# الشبكات العصبية المتكررة

في الوحدة السابقة، كنا نستخدم تمثيلات دلالية غنية للنص، ومصنفًا خطيًا بسيطًا فوق التضمينات. ما تفعله هذه البنية هو التقاط المعنى المجمع للكلمات في الجملة، لكنها لا تأخذ في الاعتبار **ترتيب** الكلمات، لأن عملية التجميع فوق التضمينات أزالت هذه المعلومات من النص الأصلي. وبسبب عدم قدرة هذه النماذج على تمثيل ترتيب الكلمات، فإنها لا تستطيع حل المهام الأكثر تعقيدًا أو الغامضة مثل توليد النصوص أو الإجابة على الأسئلة.

لتمثيل معنى تسلسل النصوص، نحتاج إلى استخدام بنية شبكة عصبية أخرى تُعرف باسم **الشبكة العصبية المتكررة** أو RNN. في RNN، نقوم بتمرير الجملة عبر الشبكة رمزًا واحدًا في كل مرة، وتنتج الشبكة **حالة** معينة، والتي نقوم بتمريرها مرة أخرى إلى الشبكة مع الرمز التالي.

بالنظر إلى تسلسل المدخلات من الرموز $X_0,\dots,X_n$، تقوم RNN بإنشاء تسلسل من كتل الشبكة العصبية، وتدرب هذا التسلسل من البداية إلى النهاية باستخدام الانتشار العكسي. كل كتلة شبكة تأخذ زوجًا $(X_i,S_i)$ كمدخل، وتنتج $S_{i+1}$ كنتيجة. الحالة النهائية $S_n$ أو المخرج $X_n$ يتم تمريرها إلى مصنف خطي لإنتاج النتيجة. جميع كتل الشبكة تشترك في نفس الأوزان، ويتم تدريبها من البداية إلى النهاية باستخدام تمريرة واحدة من الانتشار العكسي.

نظرًا لأن متجهات الحالة $S_0,\dots,S_n$ يتم تمريرها عبر الشبكة، فإنها تكون قادرة على تعلم الاعتماديات التسلسلية بين الكلمات. على سبيل المثال، عندما تظهر كلمة *ليس* في مكان ما في التسلسل، يمكنها تعلم نفي عناصر معينة داخل متجه الحالة، مما يؤدي إلى النفي.

> بما أن أوزان جميع كتل RNN في الصورة مشتركة، يمكن تمثيل نفس الصورة ككتلة واحدة (على اليمين) مع حلقة تغذية راجعة متكررة، تقوم بتمرير حالة الخرج للشبكة مرة أخرى إلى المدخل.

دعونا نرى كيف يمكن للشبكات العصبية المتكررة مساعدتنا في تصنيف مجموعة بيانات الأخبار الخاصة بنا.


In [1]:
import torch
import torchtext
from torchnlp import *
train_dataset, test_dataset, classes, vocab = load_dataset()
vocab_size = len(vocab)

Loading dataset...
Building vocab...


## مصنف RNN بسيط

في حالة RNN البسيط، كل وحدة متكررة هي شبكة خطية بسيطة، تأخذ متجه الإدخال ومتجه الحالة المدمجين، وتنتج متجه حالة جديد. يمثل PyTorch هذه الوحدة باستخدام فئة `RNNCell`، وشبكة من هذه الخلايا - كطبقة `RNN`.

لتعريف مصنف RNN، سنقوم أولاً بتطبيق طبقة تضمين لتقليل أبعاد مفردات الإدخال، ثم نضع طبقة RNN فوقها:


In [2]:
class RNNClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.rnn = torch.nn.RNN(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,h = self.rnn(x)
        return self.fc(x.mean(dim=1))

> **ملاحظة:** نستخدم هنا طبقة تضمين غير مدربة للتبسيط، ولكن للحصول على نتائج أفضل يمكننا استخدام طبقة تضمين مدربة مسبقًا باستخدام Word2Vec أو GloVe، كما هو موضح في الوحدة السابقة. لفهم أفضل، قد ترغب في تعديل هذا الكود ليعمل مع التضمينات المدربة مسبقًا.

في حالتنا، سنستخدم مُحمّل بيانات مُعبأ، بحيث يحتوي كل دفعة على عدد من التسلسلات المُعبأة بنفس الطول. ستأخذ طبقة RNN تسلسلًا من مصفوفات التضمين، وتنتج مخرجاتين:
* $x$ هو تسلسل مخرجات خلايا RNN في كل خطوة
* $h$ هو الحالة المخفية النهائية للعنصر الأخير في التسلسل

ثم نقوم بتطبيق مصنف خطي متصل بالكامل للحصول على عدد الفئات.

> **ملاحظة:** من الصعب تدريب شبكات RNN، لأنه بمجرد فك خلايا RNN على طول التسلسل، يصبح عدد الطبقات الناتجة في عملية الانتشار العكسي كبيرًا جدًا. لذلك نحتاج إلى اختيار معدل تعلم صغير، وتدريب الشبكة على مجموعة بيانات أكبر للحصول على نتائج جيدة. قد يستغرق ذلك وقتًا طويلًا، لذا يُفضل استخدام GPU.


In [3]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=padify, shuffle=True)
net = RNNClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.3090625
6400: acc=0.38921875
9600: acc=0.4590625
12800: acc=0.511953125
16000: acc=0.5506875
19200: acc=0.57921875
22400: acc=0.6070089285714285
25600: acc=0.6304296875
28800: acc=0.6484027777777778
32000: acc=0.66509375
35200: acc=0.6790056818181818
38400: acc=0.6929166666666666
41600: acc=0.7035817307692308
44800: acc=0.7137276785714286
48000: acc=0.72225
51200: acc=0.73001953125
54400: acc=0.7372794117647059
57600: acc=0.7436631944444444
60800: acc=0.7503947368421052
64000: acc=0.75634375
67200: acc=0.7615773809523809
70400: acc=0.7662642045454545
73600: acc=0.7708423913043478
76800: acc=0.7751822916666666
80000: acc=0.7790625
83200: acc=0.7825
86400: acc=0.7858564814814815
89600: acc=0.7890513392857142
92800: acc=0.7920474137931034
96000: acc=0.7952708333333334
99200: acc=0.7982258064516129
102400: acc=0.80099609375
105600: acc=0.8037594696969697
108800: acc=0.8060569852941176


## الذاكرة طويلة وقصيرة المدى (LSTM)

إحدى المشاكل الرئيسية في الشبكات العصبية المتكررة التقليدية (RNNs) هي مشكلة **تلاشي التدرجات**. نظرًا لأن الشبكات العصبية المتكررة يتم تدريبها من البداية إلى النهاية في عملية تمرير خلفي واحدة، فإنها تواجه صعوبة في نقل الخطأ إلى الطبقات الأولى من الشبكة، وبالتالي لا تستطيع الشبكة تعلم العلاقات بين الرموز البعيدة. إحدى الطرق لتجنب هذه المشكلة هي تقديم **إدارة حالة صريحة** باستخدام ما يسمى بـ **البوابات**. هناك اثنتان من أكثر البنى المعروفة من هذا النوع: **الذاكرة طويلة وقصيرة المدى** (LSTM) و**وحدة الترحيل ذات البوابة** (GRU).

![صورة توضح مثالًا على خلية ذاكرة طويلة وقصيرة المدى](../../../../../lessons/5-NLP/16-RNN/images/long-short-term-memory-cell.svg)

شبكة LSTM منظمة بطريقة مشابهة للشبكات العصبية المتكررة (RNN)، ولكن هناك حالتان يتم تمريرهما من طبقة إلى أخرى: الحالة الفعلية $c$، والمتجه المخفي $h$. في كل وحدة، يتم دمج المتجه المخفي $h_i$ مع الإدخال $x_i$، وهما يتحكمان بما يحدث للحالة $c$ عبر **البوابات**. كل بوابة هي شبكة عصبية مع تفعيل سيجمويد (الإخراج في النطاق $[0,1]$)، والتي يمكن اعتبارها كقناع ثنائي عند ضربها في متجه الحالة. هناك البوابات التالية (من اليسار إلى اليمين في الصورة أعلاه):
* **بوابة النسيان** تأخذ المتجه المخفي وتحدد أي مكونات من المتجه $c$ نحتاج إلى نسيانها وأيها نمررها.
* **بوابة الإدخال** تأخذ بعض المعلومات من الإدخال والمتجه المخفي، وتدخلها في الحالة.
* **بوابة الإخراج** تحول الحالة عبر طبقة خطية مع تفعيل $\tanh$، ثم تختار بعض مكوناتها باستخدام المتجه المخفي $h_i$ لإنتاج الحالة الجديدة $c_{i+1}$.

يمكن اعتبار مكونات الحالة $c$ كعلامات يمكن تشغيلها أو إيقافها. على سبيل المثال، عندما نصادف اسم *Alice* في التسلسل، قد نرغب في افتراض أنه يشير إلى شخصية أنثى، ونرفع العلامة في الحالة التي تشير إلى وجود اسم أنثوي في الجملة. عندما نصادف لاحقًا عبارة *and Tom*، سنرفع العلامة التي تشير إلى وجود اسم جمع. وبالتالي، من خلال التلاعب بالحالة، يمكننا نظريًا تتبع الخصائص النحوية لأجزاء الجملة.

> **Note**: مصدر رائع لفهم التفاصيل الداخلية لـ LSTM هو هذه المقالة الممتازة [Understanding LSTM Networks](https://colah.github.io/posts/2015-08-Understanding-LSTMs/) بواسطة كريستوفر أولاه.

على الرغم من أن الهيكل الداخلي لخلية LSTM قد يبدو معقدًا، إلا أن PyTorch يخفي هذه التنفيذ داخل فئة `LSTMCell`، ويوفر كائن `LSTM` لتمثيل طبقة LSTM بأكملها. وبالتالي، فإن تنفيذ مصنف LSTM سيكون مشابهًا جدًا للشبكة العصبية المتكررة البسيطة التي رأيناها أعلاه:


In [4]:
class LSTMClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x):
        batch_size = x.size(0)
        x = self.embedding(x)
        x,(h,c) = self.rnn(x)
        return self.fc(h[-1])

In [5]:
net = LSTMClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch(net,train_loader, lr=0.001)

3200: acc=0.259375
6400: acc=0.25859375
9600: acc=0.26177083333333334
12800: acc=0.2784375
16000: acc=0.313
19200: acc=0.3528645833333333
22400: acc=0.3965625
25600: acc=0.4385546875
28800: acc=0.4752777777777778
32000: acc=0.505375
35200: acc=0.5326704545454546
38400: acc=0.5557552083333334
41600: acc=0.5760817307692307
44800: acc=0.5954910714285714
48000: acc=0.6118333333333333
51200: acc=0.62681640625
54400: acc=0.6404779411764706
57600: acc=0.6520138888888889
60800: acc=0.662828947368421
64000: acc=0.673546875
67200: acc=0.6831547619047619
70400: acc=0.6917897727272727
73600: acc=0.6997146739130434
76800: acc=0.707109375
80000: acc=0.714075
83200: acc=0.7209134615384616
86400: acc=0.727037037037037
89600: acc=0.7326674107142858
92800: acc=0.7379633620689655
96000: acc=0.7433645833333333
99200: acc=0.7479032258064516
102400: acc=0.752119140625
105600: acc=0.7562405303030303
108800: acc=0.76015625
112000: acc=0.7641339285714286
115200: acc=0.7677777777777778
118400: acc=0.77112331081

(0.03487814127604167, 0.7728)

## التسلسلات المعبأة

في مثالنا، كان علينا ملء جميع التسلسلات في الدفعة الصغيرة بـ "متجهات صفرية". على الرغم من أن ذلك يؤدي إلى بعض الهدر في الذاكرة، إلا أن المشكلة الأكثر أهمية مع الشبكات العصبية المتكررة (RNNs) هي أن خلايا إضافية يتم إنشاؤها لمعالجة العناصر المعبأة، والتي تشارك في التدريب لكنها لا تحمل أي معلومات مهمة. سيكون من الأفضل تدريب الشبكة العصبية المتكررة فقط على حجم التسلسل الفعلي.

لتحقيق ذلك، يتم تقديم صيغة خاصة لتخزين التسلسلات المعبأة في PyTorch. لنفترض أن لدينا دفعة صغيرة معبأة تبدو كالتالي:
```
[[1,2,3,4,5],
 [6,7,8,0,0],
 [9,0,0,0,0]]
```
هنا يمثل الرقم 0 القيم المعبأة، ومتجه الطول الفعلي للتسلسلات المدخلة هو `[5,3,1]`.

للتدريب الفعّال للشبكة العصبية المتكررة مع التسلسلات المعبأة، نريد أن نبدأ تدريب المجموعة الأولى من خلايا الشبكة العصبية المتكررة مع دفعة صغيرة كبيرة (`[1,6,9]`)، ثم إنهاء معالجة التسلسل الثالث، ومواصلة التدريب مع دفعات صغيرة أقصر (`[2,7]`, `[3,8]`)، وهكذا. وبالتالي، يتم تمثيل التسلسل المعبأ كمتجه واحد - في حالتنا `[1,6,9,2,7,3,8,4,5]`، ومتجه الطول (`[5,3,1]`)، والذي يمكننا من خلاله إعادة بناء الدفعة الصغيرة الأصلية بسهولة.

لإنتاج تسلسل معبأ، يمكننا استخدام وظيفة `torch.nn.utils.rnn.pack_padded_sequence`. جميع الطبقات المتكررة، بما في ذلك RNN و LSTM و GRU، تدعم التسلسلات المعبأة كمدخلات، وتنتج مخرجات معبأة، والتي يمكن فك تشفيرها باستخدام `torch.nn.utils.rnn.pad_packed_sequence`.

لكي نتمكن من إنتاج تسلسل معبأ، نحتاج إلى تمرير متجه الطول إلى الشبكة، وبالتالي نحتاج إلى وظيفة مختلفة لتحضير الدفعات الصغيرة:


In [6]:
def pad_length(b):
    # build vectorized sequence
    v = [encode(x[1]) for x in b]
    # compute max length of a sequence in this minibatch and length sequence itself
    len_seq = list(map(len,v))
    l = max(len_seq)
    return ( # tuple of three tensors - labels, padded features, length sequence
        torch.LongTensor([t[0]-1 for t in b]),
        torch.stack([torch.nn.functional.pad(torch.tensor(t),(0,l-len(t)),mode='constant',value=0) for t in v]),
        torch.tensor(len_seq)
    )

train_loader_len = torch.utils.data.DataLoader(train_dataset, batch_size=16, collate_fn=pad_length, shuffle=True)

الشبكة الفعلية ستكون مشابهة جدًا لـ `LSTMClassifier` المذكور أعلاه، ولكن تمرير `forward` سيستقبل كلًا من دفعة البيانات المبطنة (padded minibatch) ومتجه أطوال التسلسل. بعد حساب التضمين (embedding)، نقوم بحساب التسلسل المضغوط (packed sequence)، ثم نمرره إلى طبقة LSTM، وبعد ذلك نقوم بفك النتيجة مرة أخرى.

> **ملاحظة**: في الواقع، نحن لا نستخدم النتيجة المفكوكة `x`، لأننا نستخدم المخرجات من الطبقات المخفية في العمليات الحسابية التالية. لذلك، يمكننا إزالة عملية فك النتيجة تمامًا من هذا الكود. السبب في وضعها هنا هو لتسهيل تعديل هذا الكود عليك، في حال احتجت إلى استخدام مخرجات الشبكة في العمليات الحسابية المستقبلية.


In [7]:
class LSTMPackClassifier(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_class):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.embedding = torch.nn.Embedding(vocab_size, embed_dim)
        self.embedding.weight.data = torch.randn_like(self.embedding.weight.data)-0.5
        self.rnn = torch.nn.LSTM(embed_dim,hidden_dim,batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, num_class)

    def forward(self, x, lengths):
        batch_size = x.size(0)
        x = self.embedding(x)
        pad_x = torch.nn.utils.rnn.pack_padded_sequence(x,lengths,batch_first=True,enforce_sorted=False)
        pad_x,(h,c) = self.rnn(pad_x)
        x, _ = torch.nn.utils.rnn.pad_packed_sequence(pad_x,batch_first=True)
        return self.fc(h[-1])

In [8]:
net = LSTMPackClassifier(vocab_size,64,32,len(classes)).to(device)
train_epoch_emb(net,train_loader_len, lr=0.001,use_pack_sequence=True)


3200: acc=0.285625
6400: acc=0.33359375
9600: acc=0.3876041666666667
12800: acc=0.44078125
16000: acc=0.4825
19200: acc=0.5235416666666667
22400: acc=0.5559821428571429
25600: acc=0.58609375
28800: acc=0.6116666666666667
32000: acc=0.63340625
35200: acc=0.6525284090909091
38400: acc=0.668515625
41600: acc=0.6822596153846154
44800: acc=0.6948214285714286
48000: acc=0.7052708333333333
51200: acc=0.71521484375
54400: acc=0.7239889705882353
57600: acc=0.7315277777777778
60800: acc=0.7388486842105263
64000: acc=0.74571875
67200: acc=0.7518303571428572
70400: acc=0.7576988636363636
73600: acc=0.7628940217391305
76800: acc=0.7681510416666667
80000: acc=0.7728125
83200: acc=0.7772235576923077
86400: acc=0.7815393518518519
89600: acc=0.7857700892857142
92800: acc=0.7895043103448276
96000: acc=0.7930520833333333
99200: acc=0.7959072580645161
102400: acc=0.798994140625
105600: acc=0.802064393939394
108800: acc=0.8051378676470589
112000: acc=0.8077857142857143
115200: acc=0.8104600694444445
118400

(0.029785829671223958, 0.8138166666666666)

> **ملاحظة:** قد تكون قد لاحظت المعامل `use_pack_sequence` الذي نقوم بتمريره إلى وظيفة التدريب. حاليًا، تتطلب وظيفة `pack_padded_sequence` أن يكون متجه طول التسلسل على جهاز CPU، وبالتالي تحتاج وظيفة التدريب إلى تجنب نقل بيانات طول التسلسل إلى GPU أثناء التدريب. يمكنك الاطلاع على تنفيذ وظيفة `train_emb` في ملف [`torchnlp.py`](../../../../../lessons/5-NLP/16-RNN/torchnlp.py).


## الشبكات العصبية المتكررة ثنائية الاتجاه ومتعددة الطبقات

في أمثلتنا، كانت جميع الشبكات المتكررة تعمل في اتجاه واحد، من بداية التسلسل إلى نهايته. يبدو هذا طبيعيًا لأنه يشبه الطريقة التي نقرأ بها ونستمع إلى الكلام. ومع ذلك، نظرًا لأننا في العديد من الحالات العملية لدينا وصول عشوائي إلى تسلسل الإدخال، فقد يكون من المنطقي تشغيل الحساب المتكرر في كلا الاتجاهين. تُعرف هذه الشبكات باسم **الشبكات العصبية المتكررة ثنائية الاتجاه**، ويمكن إنشاؤها عن طريق تمرير المعامل `bidirectional=True` إلى منشئ RNN/LSTM/GRU.

عند التعامل مع شبكة ثنائية الاتجاه، سنحتاج إلى متجهين للحالة المخفية، واحد لكل اتجاه. يقوم PyTorch بترميز هذه المتجهات كمتجه واحد بحجم مضاعف، وهو أمر مريح للغاية، لأنك عادةً ما تمرر الحالة المخفية الناتجة إلى طبقة خطية متصلة بالكامل، وستحتاج فقط إلى أخذ هذه الزيادة في الحجم في الاعتبار عند إنشاء الطبقة.

الشبكة المتكررة، سواء كانت أحادية الاتجاه أو ثنائية الاتجاه، تلتقط أنماطًا معينة داخل التسلسل، ويمكنها تخزينها في متجه الحالة أو تمريرها إلى الإخراج. كما هو الحال مع الشبكات الالتفافية، يمكننا بناء طبقة متكررة أخرى فوق الطبقة الأولى لالتقاط أنماط ذات مستوى أعلى، مبنية من الأنماط ذات المستوى الأدنى التي استخرجتها الطبقة الأولى. يقودنا هذا إلى مفهوم **الشبكات العصبية المتكررة متعددة الطبقات**، والتي تتكون من شبكتين أو أكثر من الشبكات المتكررة، حيث يتم تمرير إخراج الطبقة السابقة إلى الطبقة التالية كمدخل.

![صورة توضح شبكة عصبية متكررة طويلة وقصيرة المدى متعددة الطبقات](../../../../../translated_images/ar/multi-layer-lstm.dd975e29bb2a59fe.webp)

*الصورة مأخوذة من [هذا المقال الرائع](https://towardsdatascience.com/from-a-lstm-cell-to-a-multilayer-lstm-network-with-pytorch-2899eb5696f3) بقلم Fernando López*

يجعل PyTorch بناء مثل هذه الشبكات مهمة سهلة، حيث تحتاج فقط إلى تمرير المعامل `num_layers` إلى منشئ RNN/LSTM/GRU لبناء عدة طبقات من التكرار تلقائيًا. وهذا يعني أيضًا أن حجم متجه الحالة المخفية سيزداد بشكل متناسب، وستحتاج إلى أخذ ذلك في الاعتبار عند التعامل مع إخراج الطبقات المتكررة.


## الشبكات العصبية المتكررة لمهام أخرى

في هذه الوحدة، رأينا أن الشبكات العصبية المتكررة يمكن استخدامها لتصنيف التسلسلات، ولكن في الواقع، يمكنها التعامل مع العديد من المهام الأخرى، مثل توليد النصوص، ترجمة النصوص، والمزيد. سنناقش هذه المهام في الوحدة التالية.



---

**إخلاء المسؤولية**:  
تم ترجمة هذا المستند باستخدام خدمة الترجمة بالذكاء الاصطناعي [Co-op Translator](https://github.com/Azure/co-op-translator). بينما نسعى لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو معلومات غير دقيقة. يجب اعتبار المستند الأصلي بلغته الأصلية المصدر الرسمي. للحصول على معلومات حاسمة، يُوصى بالاستعانة بترجمة بشرية احترافية. نحن غير مسؤولين عن أي سوء فهم أو تفسيرات خاطئة تنشأ عن استخدام هذه الترجمة.
